In [1]:
from sodapy import Socrata
from dotenv import load_dotenv
from os import environ
from pathlib import Path
import datetime as dt
import pandas as pd

load_dotenv()

True

## Cleaning

In [2]:
def get_df():
    domain_src = 'data.cityofnewyork.us'
    resource_id = '43nn-pn8j'

    years = 7
    dateLimit = (dt.datetime.now() - dt.timedelta(days = years * 365)).date().isoformat()
    client = Socrata(domain_src, environ.get('nyc_open_key'))
    
    select_clause = (
        'camis,'
        'boro,'
        'zipcode,'
        'cuisine_description AS cuisine,'
        'inspection_date,'
        'inspection_type,'
        'action,'
        'violation_code,'
        'critical_flag,'
        'score,'
        'census_tract,'
        'nta,'
        'latitude,'
        'longitude'
    )

    where_clause = f'inspection_date > "{dateLimit}" AND cuisine IS NOT NULL'

    return pd.DataFrame.from_records(
        client.get(
            resource_id,
            select = select_clause,
            where = where_clause,
            limit = int(1e6)
        )
    )

In [4]:
raw_file = Path('resources/spatial_raw_open_nyc.csv')

if raw_file.exists():
    df = pd.read_csv(raw_file)
else:
    df = get_df()
    df.to_csv(raw_file, header = True, index = False)

In [5]:
action_map = {
    'Violations were cited in the following area(s).': 'cited_violation',
    'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.': 'cited_violations_and_closed',
    'No violations were recorded at the time of this inspection.': 'no_violations',
    'Establishment re-opened by DOHMH.': 'reopened',
    'Establishment re-closed by DOHMH.': 'reclosed'
}

df['action'] = df['action'].map(action_map)

In [7]:
new_cols = ['inspection_type', 'inspection_subtype']

df[new_cols] = (
    df['inspection_type']
    .str
        .split('/', n = 1, expand = True)
    .rename(
        columns = {0: new_cols[0], 1: new_cols[1]}
    )
)

for col in new_cols:
    df[col] = df[col].str.strip()


cols = list(df.columns)
new_order = cols[:6]
new_order.append(cols[-1])
new_order.extend(cols[6:-1])

df = df[new_order]

In [8]:
mask = (df['critical_flag'] == 'Not Applicable', 'score')
df.loc[mask] = df.loc[mask].fillna(int(0))

mask = (df['action'] == 'no_violations', 'score')
df.loc[mask] = df.loc[mask].fillna(int(0))

mask = (df['violation_code'].isna())
df.loc[mask, 'violation_code'] = df.loc[mask, 'violation_code'].fillna(str('None'))
df.loc[mask, 'score'] = df.loc[mask, 'score'].fillna(int(0))

In [9]:
df.dropna(how = 'any', inplace = True)

int_cols = ['camis', 'zipcode', 'score', 'census_tract']
df[int_cols] = df[int_cols].astype(int)

date_cols = ['inspection_date']
df[date_cols] = df[date_cols].apply(pd.to_datetime)

In [10]:
df.to_csv('spatial_cleaned_inspections.csv', header = True, index = False)

## Preprocessing

## Postgres Connection Stuff

In [21]:
from core.db_util import Database
from core import get_session_factory, get_settings, get_engine
from schemas import Inspection

In [22]:
db = Database(get_session_factory())

In [23]:
from sqlalchemy.dialects.postgresql import insert

In [24]:
with db.get_session() as session:
    rows = df.to_dict('records')
    stmt = insert(Inspection).values(rows)
    stmt = stmt.on_conflict_do_nothing(
        constraint = 'uq_inspection_natural'
    )
    session.execute(stmt)